In [1]:
import requests
import re

In [2]:
url_template = "https://rcijeux.fr/drupal_game/ouestfrancejeux/mfleches/grids/mfleches_%(level)d_%(index)03d.mfj"

In [3]:
data = []
for level in [1, 2, 3]:
    index = 1
    while True:
        print(level, index)
        response = requests.get(url_template % {"level": level, "index": index})
        if response.status_code == 200:
            data.append(response.text)
            index += 1
        else:
            break

3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
3 22
3 23
3 24
3 25
3 26
3 27
3 28
3 29
3 30
3 31
3 32
3 33
3 34
3 35
3 36
3 37
3 38
3 39
3 40
3 41
3 42
3 43
3 44
3 45
3 46
3 47
3 48
3 49
3 50
3 51


In [4]:
defs = {}

for string in data:

    match = re.search(r"grille:\[(.*?)\]", re.sub("[\r\n]", " ", string))
    if match is None:
        continue
    grid = [
        row.strip()[1:-1]
        for row in match.group(1).split(",")
    ]
    definitions = []
    match = re.search(r"definitions:\[(.*?)\], *spountzV", re.sub("[\r\n]", " ", string))
    for definition_string in re.split("\] *, *\[", match.group(1).strip()[1:-1]):
        definitions.append(re.sub("- *", "-", re.sub("– *", "", " ".join(map(lambda s: s.strip()[1:-1], definition_string.split(","))))))
    
    words = []
    for i, row in enumerate(grid):
        for j, cell in enumerate(row):
            if (i == 0 or re.match("[a-z]", grid[i-1][j])) and not re.match("[a-z]", cell):
                k = i
                word = ""
                while k < len(grid) and not re.match("[a-z]", grid[k][j]):
                    word += grid[k][j]
                    k += 1
                if len(word) > 1:
                    words.append(((i, j), "down", word))
            if (j == 0 or re.match("[a-z]", grid[i][j - 1])) and not re.match("[a-z]", cell):
                k = j
                word = ""
                while k < len(grid[0]) and not re.match("[a-z]", grid[i][k]):
                    word += grid[i][k]
                    k += 1
                if len(word) > 1:
                    words.append(((i, j), "accross", word))

    start_index = 0
    associations = {}
    for i, row in enumerate(grid):
        for j, cell in enumerate(row):
            if re.match("[a-z]", cell):
                targets = []
                if cell == "a":
                    targets.append((i, j + 1, "accross"))
                elif cell == "b":
                    targets.append((i + 1, j, "down"))
                elif cell == "c":
                    targets.append((i, j + 1, "down"))
                elif cell == "d":
                    targets.append((i + 1, j, "accross"))
                elif cell in "efghi":
                    targets.append((i, j + 1, "accross"))
                    targets.append((i + 1, j, "down"))
                elif cell in "jklmn":
                    targets.append((i, j + 1, "down"))
                    targets.append((i + 1, j, "down"))
                elif cell in "opqrs":
                    targets.append((i, j + 1, "accross"))
                    targets.append((i + 1, j, "accross"))
                elif cell in "tuvwx":
                    targets.append((i, j + 1, "down"))
                    targets.append((i + 1, j, "accross"))
                else:
                    raise ValueError(cell)
                for ti, tj, tdir in targets:
                    for ((wi, wj), wdir, label) in words:
                        if wi == ti and wj == tj and tdir == wdir:
                            associations[label] = definitions[start_index]
                            start_index += 1
                            break

    for word, definition in associations.items():
        defs.setdefault(word, set())
        defs[word].add(definition)


In [5]:
with open("ouestfrance.tsv", "w", encoding="utf8") as file:
    for key in sorted(defs):
        file.write("%s\t%s\n" % (
            key,
            "\t".join(defs[key])
        ))